In [11]:
import os
import random
import shutil
from PIL import Image
import torchvision.transforms as transforms

In [21]:
import os
import random
import shutil
from PIL import Image
from torchvision import transforms

In [23]:


def balance_dataset(train_dir, target_size=4000):
    """
    Balance the dataset by:
    1. Randomly removing excess images from overrepresented classes
    2. Augmenting underrepresented classes
    """
    # Get list of subdirectories (emotion categories)
    categories = [category for category in os.listdir(train_dir)
                  if os.path.isdir(os.path.join(train_dir, category))]
    
    # Count images in each category
    image_counts = {category: len(os.listdir(os.path.join(train_dir, category))) for category in categories}
    
    # Print initial image counts
    print("Initial Image Counts:")
    for category, count in image_counts.items():
        print(f"{category}: {count} images")
    
    # Define augmentation transformations
    transform = transforms.Compose([
        transforms.RandomRotation(10),           # Random rotation
        transforms.RandomHorizontalFlip(),       # Random horizontal flip
        transforms.RandomVerticalFlip(),         # Random vertical flip
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Random color jitter
        transforms.RandomAffine(degrees=5, translate=(0.1, 0.1))  # Random affine transform
    ])
    
    # Create a backup of the original dataset
    backup_dir = train_dir + "_backup"
    shutil.copytree(train_dir, backup_dir, dirs_exist_ok=True)
    
    # Process each category
    for category in categories:
        category_path = os.path.join(train_dir, category)
        images = os.listdir(category_path)
        
        if len(images) > target_size:
            # If category has more than target size, randomly remove excess images
            excess_images = random.sample(images, len(images) - target_size)
            for img in excess_images:
                os.remove(os.path.join(category_path, img))
            print(f"Removed {len(excess_images)} images from {category}")
        
        elif len(images) < target_size:
            # If category has fewer than target size, augment images
            augmented_images_needed = target_size - len(images)
            augmented_count = 0
            
            while augmented_count < augmented_images_needed:
                # Randomly select an image to augment
                source_image = random.choice(images)
                source_path = os.path.join(category_path, source_image)
                
                # Open the image
                img = Image.open(source_path)
                
                # Apply transformations
                augmented_img = transform(img)
                
                # Generate a unique filename
                base_name = os.path.splitext(source_image)[0]
                augmented_img_name = f"{base_name}_aug_{augmented_count}.jpg"
                augmented_img_path = os.path.join(category_path, augmented_img_name)
                
                # Save the augmented image
                augmented_img.save(augmented_img_path)
                augmented_count += 1
            
            print(f"Augmented {augmented_count} images for {category}")
    
    # Verify final counts
    final_counts = {category: len(os.listdir(os.path.join(train_dir, category))) for category in categories}
    print("\nFinal Image Counts:")
    for category, count in final_counts.items():
        print(f"{category}: {count} images")


# Set the path to your image dataset
train_dir = r"dataset\images\train"  # Use a raw string to avoid escape sequence errors

# Balance the dataset
balance_dataset(train_dir)


Initial Image Counts:
angry: 3993 images
disgust: 436 images
fear: 4103 images
happy: 7164 images
neutral: 4982 images
sad: 4938 images
surprise: 3205 images
Augmented 7 images for angry
Augmented 3564 images for disgust
Removed 103 images from fear
Removed 3164 images from happy
Removed 982 images from neutral
Removed 938 images from sad
Augmented 795 images for surprise

Final Image Counts:
angry: 4000 images
disgust: 4000 images
fear: 4000 images
happy: 4000 images
neutral: 4000 images
sad: 4000 images
surprise: 4000 images


In [25]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11s-cls.yaml")  # build a new model from YAML
model = YOLO("yolo11s-cls.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo11s-cls.yaml").load("yolo11s-cls.pt")  # build from YAML and transfer weights

# Train the model
results = model.train(data=r"dataset\images", epochs=50, imgsz=64)

YOLO11s-cls summary: 151 layers, 5,545,488 parameters, 5,545,488 gradients, 12.2 GFLOPs


100%|██████████| 13.0M/13.0M [00:00<00:00, 82.9MB/s]


YOLO11s-cls summary: 151 layers, 5,545,488 parameters, 5,545,488 gradients, 12.2 GFLOPs
Transferred 234/236 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.99 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.56  Python-3.12.9 torch-2.5.1+cpu CPU (AMD Ryzen Threadripper 3960X 24-Core Processor)
engine\trainer: task=classify, mode=train, model=yolo11s-cls.yaml, data=dataset\images, epochs=50, time=None, patience=100, batch=16, imgsz=64, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train12, exist_ok=False, pretrained=yolo11s-cls.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, sourc

train: Scanning C:\work\alma\Facial-Emotion-Recognition\dataset\images\train... 28000 images, 0 corrupt: 100%|██████████| 28000/28000 [00:00<?, ?it/s]
val: Scanning C:\work\alma\Facial-Emotion-Recognition\dataset\images\test... 11929 images, 1 corrupt: 100%|██████████| 11929/11929 [00:00<?, ?it/s]

val: WARNING  C:\work\alma\Facial-Emotion-Recognition\dataset\images\test\disgust\21744_aug_1823.jpg: ignoring corrupt image/label: cannot identify image file 'C:\\work\\alma\\Facial-Emotion-Recognition\\dataset\\images\\test\\disgust\\21744_aug_1823.jpg'
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
Image sizes 64 train, 64 val
Using 0 dataloader workers
Logging results to runs\classify\train12
Starting training for 50 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

      50/50         0G     0.7961         16         64: 100%|██████████| 1750/1750 [02:42<00:00, 10.77it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 373/373 [00:14<00:00, 26.02it/s]

                   all      0.559      0.986



50 epochs completed in 2.702 hours.
Optimizer stripped from runs\classify\train12\weights\last.pt, 11.0MB
Optimizer stripped from runs\classify\train12\weights\best.pt, 11.0MB

Validating runs\classify\train12\weights\best.pt...
Ultralytics 8.3.56  Python-3.12.9 torch-2.5.1+cpu CPU (AMD Ryzen Threadripper 3960X 24-Core Processor)
YOLO11s-cls summary (fused): 112 layers, 5,443,095 parameters, 0 gradients, 12.0 GFLOPs
WARNING  Dataset 'split=val' not found, using 'split=test' instead.
train: C:\work\alma\Facial-Emotion-Recognition\dataset\images\train... found 28000 images in 7 classes  
val: None...
test: C:\work\alma\Facial-Emotion-Recognition\dataset\images\test... found 11930 images in 7 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 373/373 [00:13<00:00, 27.85it/s]


                   all       0.56      0.986
Speed: 0.0ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train12


In [5]:
from ultralytics import YOLO
model = YOLO("yolo11s_emotion.pt")
results = model("unseen_data/happy_pramod.jpg")
results[0].show()


image 1/1 /mnt/c/work/alma/Facial-Emotion-Recognition/unseen_data/happy_pramod.jpg: 224x224 happy 0.83, neutral 0.07, sad 0.06, angry 0.03, fear 0.00, 6.4ms
Speed: 29.1ms preprocess, 6.4ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)


Try running the update-desktop-database command. If you
don't have this command you should install the
desktop-file-utils package. This package is available from
http://freedesktop.org/wiki/Software/desktop-file-utils/
No applications found for mimetype: image/png
./usr/bin/xdg-open: 882: x-www-browser: not found
/usr/bin/xdg-open: 882: firefox: not found
/usr/bin/xdg-open: 882: iceweasel: not found
/usr/bin/xdg-open: 882: seamonkey: not found
/usr/bin/xdg-open: 882: mozilla: not found
/usr/bin/xdg-open: 882: epiphany: not found
/usr/bin/xdg-open: 882: konqueror: not found
/usr/bin/xdg-open: 882: chromium: not found
/usr/bin/xdg-open: 882: chromium-browser: not found
/usr/bin/xdg-open: 882: google-chrome: not found
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method avai

In [1]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11l-cls.yaml")  # build a new model from YAML
model = YOLO("yolo11l-cls.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo11l-cls.yaml").load("yolo11l-cls.pt")  # build from YAML and transfer weights

# Train the model
# results = model.train(data=r"dataset\images", epochs=25, imgsz=64,device=0)

results = model.train(data="./dataset/images", epochs=25, imgsz=64,device=0)

FlashAttention is not available on this device. Using scaled_dot_product_attention instead.


/root/.pyenv/versions/core-backend-py3.10/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


YOLO11l-cls summary: 309 layers, 12,937,104 parameters, 12,937,104 gradients, 54.1 GFLOPs
YOLO11l-cls summary: 309 layers, 12,937,104 parameters, 12,937,104 gradients, 54.1 GFLOPs
Transferred 492/494 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.113 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.63 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
engine/trainer: task=classify, mode=train, model=yolo11l-cls.yaml, data=./dataset/images, epochs=25, time=None, patience=100, batch=16, imgsz=64, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train3, exist_ok=False, pretrained=yolo11l-cls.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hy

E0000 00:00:1745302151.266974    2905 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745302151.289408    2905 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745302151.460125    2905 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745302151.460146    2905 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745302151.460148    2905 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745302151.460149    2905 computation_placer.cc:177] computation placer already registered. Please check linka

Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  2    173824  ultralytics.nn.modules.block.C3k2            [128, 256, 2, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  2    691712  ultralytics.nn.modules.block.C3k2            [256, 512, 2, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  2   2234368  ultralytics.nn.modules.block.C3k2            [512, 512, 2, True]           
  7                  -1  1   2360320  ultralytics

...

      25/25     0.776G     0.8246         16         64: 100%|██████████| 1750/1750 [02:17<00:00, 12.75it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 373/373 [00:12<00:00, 29.66it/s]

                   all      0.561      0.991



25 epochs completed in 0.977 hours.
Optimizer stripped from runs/classify/train3/weights/last.pt, 25.9MB
Optimizer stripped from runs/classify/train3/weights/best.pt, 25.9MB

Validating runs/classify/train3/weights/best.pt...
Ultralytics 8.3.63 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
YOLO11l-cls summary (fused): 227 layers, 12,827,399 parameters, 0 gradients, 53.5 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/train... found 28000 images in 7 classes ✅ 
val: None...
test: /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/test... found 11930 images in 7 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 373/373 [00:11<00:00, 31.62it/s]


                   all       0.56      0.991
Speed: 0.0ms preprocess, 0.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train3


In [1]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11x-cls.yaml")  # build a new model from YAML
model = YOLO("yolo11x-cls.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo11x-cls.yaml").load("yolo11x-cls.pt")  # build from YAML and transfer weights

# Train the model
# results = model.train(data=r"dataset\images", epochs=25, imgsz=64,device=0)

results = model.train(data="./dataset/images", epochs=50, imgsz=64,device=0)

FlashAttention is not available on this device. Using scaled_dot_product_attention instead.


/root/.pyenv/versions/core-backend-py3.10/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


YOLO11x-cls summary: 309 layers, 28,458,544 parameters, 28,458,544 gradients, 117.4 GFLOPs
YOLO11x-cls summary: 309 layers, 28,458,544 parameters, 28,458,544 gradients, 117.4 GFLOPs
Transferred 492/494 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.113 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.63 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
engine/trainer: task=classify, mode=train, model=yolo11x-cls.yaml, data=./dataset/images, epochs=50, time=None, patience=100, batch=16, imgsz=64, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train5, exist_ok=False, pretrained=yolo11x-cls.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_

E0000 00:00:1745308827.258017    1513 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745308827.279961    1513 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745308827.451916    1513 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745308827.451936    1513 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745308827.451938    1513 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745308827.451939    1513 computation_placer.cc:177] computation placer already registered. Please check linka

Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1      2784  ultralytics.nn.modules.conv.Conv             [3, 96, 3, 2]                 
  1                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  2                  -1  2    389760  ultralytics.nn.modules.block.C3k2            [192, 384, 2, True, 0.25]     
  3                  -1  1   1327872  ultralytics.nn.modules.conv.Conv             [384, 384, 3, 2]              
  4                  -1  2   1553664  ultralytics.nn.modules.block.C3k2            [384, 768, 2, True, 0.25]     
  5                  -1  1   5309952  ultralytics.nn.modules.conv.Conv             [768, 768, 3, 2]              
  6                  -1  2   5022720  ultralytics.nn.modules.block.C3k2            [768, 768, 2, True]           
  7                  -1  1   5309952  ultralytics

...


      Epoch    GPU_mem       loss  Instances       Size


      39/50      1.07G     0.8068         16         64: 100%|██████████| 1750/1750 [02:01<00:00, 14.39it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 373/373 [00:12<00:00, 30.26it/s]


                   all      0.565      0.989

      Epoch    GPU_mem       loss  Instances       Size


      40/50      0.94G     0.7739         16         64:  10%|▉         | 172/1750 [00:11<01:45, 14.93it/s]


KeyboardInterrupt: 

In [3]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8x-cls.yaml")  # build a new model from YAML
model = YOLO("yolov8x-cls.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolov8x-cls.yaml").load("yolov8x-cls.pt")  # build from YAML and transfer weights

# Train the model
# results = model.train(data=r"dataset\images", epochs=25, imgsz=64,device=0)

results = model.train(data="./dataset/images", epochs=50, imgsz=64,device=0)

YOLOv8x-cls summary: 183 layers, 57,422,840 parameters, 57,422,840 gradients, 165.9 GFLOPs


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 110M/110M [00:10<00:00, 11.4MB/s]


YOLOv8x-cls summary: 183 layers, 57,422,840 parameters, 57,422,840 gradients, 165.9 GFLOPs
Transferred 302/302 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.113 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.63 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
engine/trainer: task=classify, mode=train, model=yolov8x-cls.yaml, data=./dataset/images, epochs=50, time=None, patience=100, batch=16, imgsz=64, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train6, exist_ok=False, pretrained=yolov8x-cls.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=No

train: Scanning /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/train... 28000 images, 0 corrupt: 100%|██████████| 28000/28000 [00:00<?, ?it/s]
val: Scanning /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/test... 11929 images, 1 corrupt: 100%|██████████| 11929/11929 [00:00<?, ?it/s]

val: /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/test/disgust/21744_aug_1823.jpg: ignoring corrupt image/label: cannot identify image file '/mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/test/disgust/21744_aug_1823.jpg'


...


      Epoch    GPU_mem       loss  Instances       Size


      50/50       1.8G     0.4249         16         64: 100%|██████████| 1750/1750 [01:25<00:00, 20.54it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 373/373 [00:14<00:00, 25.43it/s]


                   all      0.579      0.974

50 epochs completed in 1.397 hours.
Optimizer stripped from runs/classify/train6/weights/last.pt, 112.5MB
Optimizer stripped from runs/classify/train6/weights/best.pt, 112.5MB

Validating runs/classify/train6/weights/best.pt...
Ultralytics 8.3.63 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
YOLOv8x-cls summary (fused): 133 layers, 56,132,167 parameters, 0 gradients, 164.3 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/train... found 28000 images in 7 classes ✅ 
val: None...
test: /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/test... found 11930 images in 7 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 373/373 [00:12<00:00, 29.50it/s]


                   all       0.57      0.986
Speed: 0.0ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train6


In [5]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8s-cls.yaml")  # build a new model from YAML
model = YOLO("yolov8s-cls.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolov8s-cls.yaml").load("yolov8s-cls.pt")  # build from YAML and transfer weights

# Train the model
# results = model.train(data=r"dataset\images", epochs=25, imgsz=64,device=0)

results = model.train(data="./dataset/images", epochs=50, imgsz=64, batch=32 , device=0, dropout = 0.2)

YOLOv8s-cls summary: 99 layers, 6,361,736 parameters, 6,361,736 gradients, 17.8 GFLOPs
YOLOv8s-cls summary: 99 layers, 6,361,736 parameters, 6,361,736 gradients, 17.8 GFLOPs
Transferred 158/158 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.113 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.63 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
engine/trainer: task=classify, mode=train, model=yolov8s-cls.yaml, data=./dataset/images, epochs=50, time=None, patience=100, batch=32, imgsz=64, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train8, exist_ok=False, pretrained=yolov8s-cls.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=F

train: Scanning /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/train... 28000 images, 0 corrupt: 100%|██████████| 28000/28000 [00:00<?, ?it/s]
val: Scanning /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/test... 11929 images, 1 corrupt: 100%|██████████| 11929/11929 [00:00<?, ?it/s]

val: /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/test/disgust/21744_aug_1823.jpg: ignoring corrupt image/label: cannot identify image file '/mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/test/disgust/21744_aug_1823.jpg'


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 64 train, 64 val
Using 8 dataloader workers
Logging results to runs/classify/train8
Starting training for 50 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 187/187 [00:12<00:00, 14.44it/s]

...

      50/50      1.31G     0.6809         32         64: 100%|██████████| 875/875 [00:34<00:00, 25.27it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 187/187 [00:12<00:00, 15.04it/s]

                   all      0.562      0.985



50 epochs completed in 0.656 hours.
Optimizer stripped from runs/classify/train8/weights/last.pt, 10.3MB
Optimizer stripped from runs/classify/train8/weights/best.pt, 10.3MB

Validating runs/classify/train8/weights/best.pt...
Ultralytics 8.3.63 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
YOLOv8s-cls summary (fused): 73 layers, 5,084,167 parameters, 0 gradients, 16.7 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/train... found 28000 images in 7 classes ✅ 
val: None...
test: /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/test... found 11930 images in 7 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 187/187 [00:10<00:00, 18.14it/s]


                   all      0.562      0.986
Speed: 0.1ms preprocess, 0.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train8
